## Import Libraries

In [68]:
import tensorflow as tf
import numpy as np
import pandas as pd
import seaborn as sns
from tensorflow.keras.layers import IntegerLookup
from tensorflow.keras.layers import Normalization
from tensorflow.keras.layers import StringLookup
import tensorflow.keras as keras

## Load The Data to DataFrame

In [69]:
df = pd.read_csv('cleaned_data.csv')

## Check the Columns' Data Type

In [70]:
df.columns

Index(['gender', 'age', 'workout_experience', 'workout_time', 'weight',
       'height', 'fitness_goal'],
      dtype='object')

In [107]:
from sklearn.utils import shuffle

shuffled_df = shuffle(df, random_state = 42)

## Split Data for Training and Validation


In [108]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(shuffled_df, test_size = 0.2, random_state=42)

## The training and validation set size

In [109]:
print('training : ',train_df.shape[0], " | validation : ", val_df.shape[0])

training :  723  | validation :  181


## Function Transform Dataframe into Tensor Dataset

In [110]:
def dataframe_transform(df, label):

    labels = df.pop(label)
    ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
    ds = ds.shuffle(buffer_size = len(df))

    return ds

## Transform Training and Validation Dataframe into Tensor Dataset

In [111]:
train_ds = dataframe_transform(train_df, 'fitness_goal')
val_ds   = dataframe_transform(val_df, 'fitness_goal')

## Batching the Dataset

In [112]:
train_ds = train_ds.batch(20)
val_ds   = val_ds.batch(20)

## Function to Encode Categorical Column

In [113]:
def encode_categorical_feature(column, name, dataset):

    lookup_class = IntegerLookup
    lookup = lookup_class(output_mode = "binary")

    column_ds = dataset.map(lambda x, y: x[name])
    column_ds = column_ds.map(lambda x: tf.expand_dims(x, -1))

    lookup.adapt(column_ds)
    encoded_column = lookup(column)

    return encoded_column

## Function to Encode Numerical Column

In [114]:
def encode_numerical_feature(column, name, dataset):

    normalizer = Normalization()
    column_ds = dataset.map(lambda x, y: x[name])
    column_ds = column_ds.map(lambda x: tf.expand_dims(x, -1))

    normalizer.adapt(column_ds)
    encoded_column = normalizer(column)

    return encoded_column

## Creating Tensor object from the column

In [115]:
gender = keras.Input(shape = (1,), name = 'gender', dtype = 'int64')
age    = keras.Input(shape = (1,), name = 'age')
workout_experience = keras.Input(shape = (1, ), name = 'workout_experience')
workout_time = keras.Input(shape = (1,), name = 'workout_time')
weight       = keras.Input(shape = (1,), name = 'weight')
height       = keras.Input(shape = (1,), name = 'height')

In [116]:
all_columns = [ gender, age, workout_experience, workout_time, height, weight ]

## Encode columns based on their data type

In [117]:
gender_encoded = encode_categorical_feature(gender, 'gender', train_ds)
age_encoded = encode_numerical_feature(age, 'age', train_ds)
workout_experience_encoded = encode_numerical_feature(workout_experience, 'workout_experience', train_ds)
workout_time_encoded = encode_numerical_feature(workout_time, 'workout_time', train_ds)
weight_encoded = encode_numerical_feature(weight, 'weight', train_ds)
height_encoded = encode_numerical_feature(height, 'height', train_ds)


In [118]:
all_features = keras.layers.concatenate([gender_encoded, age_encoded, workout_experience_encoded, workout_time_encoded, height_encoded, weight_encoded])

## Build the model architecture

In [119]:
x = keras.layers.Dense(64, activation = 'relu')(all_features)
x = keras.layers.Dense(16, activation = 'relu')(x)
x = keras.layers.Dropout(0.7)(x)

output = keras.layers.Dense(1, activation = 'sigmoid')(x)

In [120]:
model  = keras.Model(all_columns, output)
model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['accuracy'])

## Train the model

In [121]:
model.fit(train_ds, epochs=300, validation_data = val_ds)

Epoch 1/300
37/37 [==============================] - 2s 12ms/step - loss: 0.7150 - accuracy: 0.5187 - val_loss: 0.6872 - val_accuracy: 0.5746
Epoch 2/300
37/37 [==============================] - 0s 3ms/step - loss: 0.7026 - accuracy: 0.5228 - val_loss: 0.6835 - val_accuracy: 0.6077
Epoch 3/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6943 - accuracy: 0.5353 - val_loss: 0.6857 - val_accuracy: 0.5691
Epoch 4/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6983 - accuracy: 0.5187 - val_loss: 0.6842 - val_accuracy: 0.5856
Epoch 5/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6916 - accuracy: 0.5436 - val_loss: 0.6849 - val_accuracy: 0.5525
Epoch 6/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6880 - accuracy: 0.5353 - val_loss: 0.6851 - val_accuracy: 0.5691
Epoch 7/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6852 - accuracy: 0.5671 - val_loss: 0.6858 - val_accuracy: 0.5525
Epoch 8/300


Epoch 59/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6699 - accuracy: 0.5712 - val_loss: 0.6903 - val_accuracy: 0.5691
Epoch 60/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6648 - accuracy: 0.5920 - val_loss: 0.6882 - val_accuracy: 0.5580
Epoch 61/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6600 - accuracy: 0.5823 - val_loss: 0.6853 - val_accuracy: 0.5635
Epoch 62/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6672 - accuracy: 0.5851 - val_loss: 0.6883 - val_accuracy: 0.5967
Epoch 63/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6613 - accuracy: 0.5864 - val_loss: 0.6875 - val_accuracy: 0.5801
Epoch 64/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6664 - accuracy: 0.5754 - val_loss: 0.6894 - val_accuracy: 0.5967
Epoch 65/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6730 - accuracy: 0.5657 - val_loss: 0.6886 - val_accuracy: 0.5801
Epoch 

37/37 [==============================] - 0s 3ms/step - loss: 0.6481 - accuracy: 0.6169 - val_loss: 0.6998 - val_accuracy: 0.5470
Epoch 117/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6452 - accuracy: 0.5892 - val_loss: 0.7015 - val_accuracy: 0.5193
Epoch 118/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6521 - accuracy: 0.5795 - val_loss: 0.6966 - val_accuracy: 0.5359
Epoch 119/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6460 - accuracy: 0.5947 - val_loss: 0.6981 - val_accuracy: 0.5249
Epoch 120/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6410 - accuracy: 0.6044 - val_loss: 0.7012 - val_accuracy: 0.5249
Epoch 121/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6429 - accuracy: 0.5878 - val_loss: 0.7123 - val_accuracy: 0.4972
Epoch 122/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6453 - accuracy: 0.5823 - val_loss: 0.7033 - val_accuracy: 0.5304
Epoch 123/300

37/37 [==============================] - 0s 3ms/step - loss: 0.6215 - accuracy: 0.6307 - val_loss: 0.7236 - val_accuracy: 0.4972
Epoch 174/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6344 - accuracy: 0.6100 - val_loss: 0.7301 - val_accuracy: 0.5249
Epoch 175/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6376 - accuracy: 0.5934 - val_loss: 0.7172 - val_accuracy: 0.5138
Epoch 176/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6218 - accuracy: 0.6279 - val_loss: 0.7233 - val_accuracy: 0.5359
Epoch 177/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6222 - accuracy: 0.6293 - val_loss: 0.7198 - val_accuracy: 0.5249
Epoch 178/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6293 - accuracy: 0.6141 - val_loss: 0.7163 - val_accuracy: 0.5414
Epoch 179/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6229 - accuracy: 0.6293 - val_loss: 0.7176 - val_accuracy: 0.5359
Epoch 180/300

37/37 [==============================] - 0s 3ms/step - loss: 0.6058 - accuracy: 0.6349 - val_loss: 0.7373 - val_accuracy: 0.5138
Epoch 231/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6079 - accuracy: 0.6542 - val_loss: 0.7431 - val_accuracy: 0.5193
Epoch 232/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6229 - accuracy: 0.6183 - val_loss: 0.7304 - val_accuracy: 0.5470
Epoch 233/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6122 - accuracy: 0.6266 - val_loss: 0.7335 - val_accuracy: 0.5359
Epoch 234/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5984 - accuracy: 0.6584 - val_loss: 0.7300 - val_accuracy: 0.5193
Epoch 235/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6273 - accuracy: 0.6127 - val_loss: 0.7282 - val_accuracy: 0.5083
Epoch 236/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6248 - accuracy: 0.6210 - val_loss: 0.7209 - val_accuracy: 0.4972
Epoch 237/300

37/37 [==============================] - 0s 3ms/step - loss: 0.6142 - accuracy: 0.6293 - val_loss: 0.7334 - val_accuracy: 0.5138
Epoch 288/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5947 - accuracy: 0.6376 - val_loss: 0.7477 - val_accuracy: 0.4972
Epoch 289/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6084 - accuracy: 0.6321 - val_loss: 0.7547 - val_accuracy: 0.4972
Epoch 290/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6175 - accuracy: 0.6155 - val_loss: 0.7355 - val_accuracy: 0.5193
Epoch 291/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6130 - accuracy: 0.6252 - val_loss: 0.7471 - val_accuracy: 0.5028
Epoch 292/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6037 - accuracy: 0.6362 - val_loss: 0.7489 - val_accuracy: 0.5028
Epoch 293/300
37/37 [==============================] - 0s 4ms/step - loss: 0.6157 - accuracy: 0.6252 - val_loss: 0.7351 - val_accuracy: 0.5193
Epoch 294/300